# **INF8460 A20 Project: Open-domain questions answering**

<br>

Equipe 8:


*   Cedric Sadeu
*   Mamoudou Sacko
*   Oumayma Messoussi

<br>

---

<br>

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.8MB/s 
     |████████████████████████████████| 1.1MB 28.6MB/s 
     |████████████████████████████████| 890kB 46.3MB/s 
     |████████████████████████████████| 2.9MB 50.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=30a33e73df1c9122ddd1b80af3fbe2efa1bf6c0e24dab3638044e5efa16c24a0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
import numpy as np
from transformers import pipeline, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, BertTokenizer, BertModel, BertForQuestionAnswering



---

<br>

## **1. Plongements lexicaux**

### Lecture des donnees

### TF-IDF

### GloVe

### Plongements contextuels pré-entraînés / non pré-entraînés



> #### Huggingface ready pipeline



In [25]:
question = "How many parameters does BERT-large have?"
answer_text = r"""BERT-large is really big... it has 24-layers and an embedding size of 1,024, 
                  for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take 
                  a couple minutes to download to your Colab instance."""

nlp = pipeline("question-answering")
result = nlp(question=question, context=answer_text)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1374: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


Answer: '340M', score: 0.7121, start: 111, end: 115




> #### DistilBERT SQuAD pre-trained



In [35]:
questions = ["How many parameters does BERT-large have?"]
answer_text = r"""BERT-large is really big... it has 24-layers and an embedding size of 1,024, 
                  for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take 
                  a couple minutes to download to your Colab instance."""

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad", 
                                                      return_dict=True, output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad", return_dict=True)

for question in questions:
    inputs = tokenizer(question, answer_text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # display tokens and ids
    for token, id in zip(text_tokens, input_ids):
        if id == tokenizer.sep_token_id:
            print('')
        print('{:<12} {:>6,}'.format(token, id))
        if id == tokenizer.sep_token_id:
            print('')

    outputs = model(**inputs)

    last_hidden_states = outputs.hidden_states[-1]
    print(last_hidden_states.shape)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}, start: {answer_start}, end: {answer_end}")

[CLS]           101
How           1,731
many          1,242
parameters   11,934
does          1,674
B               139
##ER          9,637
##T           1,942
-               118
large         1,415
have          1,138
?               136

[SEP]           102

B               139
##ER          9,637
##T           1,942
-               118
large         1,415
is            1,110
really        1,541
big           1,992
.               119
.               119
.               119
it            1,122
has           1,144
24            1,572
-               118
layers        8,798
and           1,105
an            1,126
em            9,712
##bed         4,774
##ding        3,408
size          2,060
of            1,104
1               122
,               117
02            5,507
##4           1,527
,               117
for           1,111
a               170
total         1,703
of            1,104
340          16,984
##M           2,107
parameters   11,934
!               106
Alto         17,76

> #### BERT base pre-trained

In [25]:
torch.cuda.set_device(0)

questions = ["How many parameters does BERT-large have?"]
answer_text = r"""BERT-large is really big... it has 24-layers and an embedding size of 1,024, 
                  for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take 
                  a couple minutes to download to your Colab instance."""

model = BertModel.from_pretrained("bert-base-cased", return_dict=True)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)
# # model = model.to(device)

for question in questions:
    inputs = tokenizer(question, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # display tokens and ids
    for token, id in zip(text_tokens, input_ids):
        if id == tokenizer.sep_token_id:
            print('')
        print('{:<12} {:>6,}'.format(token, id))
        if id == tokenizer.sep_token_id:
            print('')

    outputs = model(**inputs)

    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states.shape)

[CLS]           101
How           1,731
many          1,242
parameters   11,934
does          1,674
B               139
##ER          9,637
##T           1,942
-               118
large         1,415
have          1,138
?               136

[SEP]           102

torch.Size([1, 13, 768])


> #### DistilBERT SQuAD training